In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Stap 1: Definieer Pydantic-modellen voor gestructureerde outputs

Deze modellen definiëren de **schema** die agents zullen retourneren. Het gebruik van `response_format` met Pydantic zorgt voor:
- ✅ Type-veilige data-extractie
- ✅ Automatische validatie
- ✅ Geen parsefouten bij vrije-tekst antwoorden
- ✅ Eenvoudige conditionele routering op basis van velden


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Stap 2: Maak de Hotelboekingsmodule

Deze module wordt door de **availability_agent** gebruikt om te controleren of er kamers beschikbaar zijn. We gebruiken de `@ai_function` decorator om:
- Een Python-functie om te zetten in een AI-oproepbare tool
- Automatisch een JSON-schema te genereren voor de LLM
- Parametervalidatie af te handelen
- Automatische aanroep door agents mogelijk te maken

Voor deze demo:
- **Stockholm, Seattle, Tokyo, Londen, Amsterdam** → Kamers beschikbaar ✅
- **Alle andere steden** → Geen kamers beschikbaar ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Stap 3: Definieer Voorwaardefuncties voor Routering

Deze functies beoordelen de reactie van de agent en bepalen welke route in de workflow gevolgd moet worden.

**Belangrijk patroon:**
1. Controleer of het bericht een `AgentExecutorResponse` is
2. Analyseer de gestructureerde output (Pydantic-model)
3. Geef `True` of `False` terug om de routering te sturen

De workflow zal deze voorwaarden evalueren op **kanten** om te bepalen welke uitvoerder vervolgens moet worden aangeroepen.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Stap 4: Maak een aangepaste Display Executor

Executors zijn workflowcomponenten die transformaties of neveneffecten uitvoeren. We gebruiken de `@executor` decorator om een aangepaste executor te maken die het eindresultaat weergeeft.

**Belangrijke concepten:**
- `@executor(id="...")` - Registreert een functie als een workflowexecutor
- `WorkflowContext[Never, str]` - Type hints voor input/output
- `ctx.yield_output(...)` - Geeft het eindresultaat van de workflow terug


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Stap 5: Laad omgevingsvariabelen

Configureer de LLM-client. Dit voorbeeld werkt met:
- **GitHub-modellen** (Gratis tier met GitHub-token)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Stap 6: Maak AI-agenten met gestructureerde outputs

We maken **drie gespecialiseerde agenten**, elk verpakt in een `AgentExecutor`:

1. **availability_agent** - Controleert hotelbeschikbaarheid met behulp van de tool
2. **alternative_agent** - Stelt alternatieve steden voor (wanneer er geen kamers beschikbaar zijn)
3. **booking_agent** - Moedigt aan tot boeken (wanneer er kamers beschikbaar zijn)

**Belangrijke kenmerken:**
- `tools=[hotel_booking]` - Biedt de tool aan de agent
- `response_format=PydanticModel` - Dwingt gestructureerde JSON-output af
- `AgentExecutor(..., id="...")` - Verpakt de agent voor gebruik in workflows


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Stap 7: Bouw de workflow met conditionele verbindingen

Nu gebruiken we `WorkflowBuilder` om de grafiek te construeren met conditionele routing:

**Workflowstructuur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Belangrijke methoden:**
- `.set_start_executor(...)` - Stelt het startpunt in
- `.add_edge(from, to, condition=...)` - Voegt een conditionele verbinding toe
- `.build()` - Finaliseert de workflow


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Stap 8: Voer Testgeval 1 uit - Stad ZONDER Beschikbaarheid (Parijs)

Laten we het pad **zonder beschikbaarheid** testen door hotels in Parijs aan te vragen (die in onze simulatie geen kamers heeft).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Stap 9: Voer Testgeval 2 uit - Stad MET Beschikbaarheid (Stockholm)

Laten we nu het **beschikbaarheids**pad testen door hotels in Stockholm op te vragen (die kamers hebben in onze simulatie).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Belangrijkste Inzichten en Volgende Stappen

### ✅ Wat Je Hebt Geleerd:

1. **WorkflowBuilder-patroon**
   - Gebruik `.set_start_executor()` om het startpunt te definiëren
   - Gebruik `.add_edge(from, to, condition=...)` voor conditionele routering
   - Roep `.build()` aan om de workflow te voltooien

2. **Conditionele Routering**
   - Conditiefuncties inspecteren `AgentExecutorResponse`
   - Gestructureerde outputs analyseren om routeringsbeslissingen te nemen
   - Retourneer `True` om een edge te activeren, `False` om deze over te slaan

3. **Toolintegratie**
   - Gebruik `@ai_function` om Python-functies om te zetten in AI-tools
   - Agents roepen tools automatisch aan wanneer nodig
   - Tools retourneren JSON die door agents kan worden geanalyseerd

4. **Gestructureerde Outputs**
   - Gebruik Pydantic-modellen voor typeveilige data-extractie
   - Stel `response_format=MyModel` in bij het maken van agents
   - Analyseer reacties met `Model.model_validate_json()`

5. **Aangepaste Executors**
   - Gebruik `@executor(id="...")` om workflowcomponenten te maken
   - Executors kunnen data transformeren of neveneffecten uitvoeren
   - Gebruik `ctx.yield_output()` om workflowresultaten te produceren

### 🚀 Toepassingen in de Praktijk:

- **Reisboekingen**: Beschikbaarheid controleren, alternatieven voorstellen, opties vergelijken
- **Klantenservice**: Routeren op basis van probleemtype, sentiment, prioriteit
- **E-commerce**: Voorraad controleren, alternatieven voorstellen, bestellingen verwerken
- **Contentmoderatie**: Routeren op basis van toxiciteitsscores, gebruikersmeldingen
- **Goedkeuringsworkflows**: Routeren op basis van bedrag, gebruikersrol, risiconiveau
- **Meerstapsverwerking**: Routeren op basis van datakwaliteit, volledigheid

### 📚 Volgende Stappen:

- Complexere voorwaarden toevoegen (meerdere criteria)
- Lussen implementeren met workflowstatusbeheer
- Subworkflows toevoegen voor herbruikbare componenten
- Integreren met echte API's (hotelboekingen, voorraadsystemen)
- Foutafhandeling en alternatieve paden toevoegen
- Workflows visualiseren met de ingebouwde visualisatietools



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
